In [1]:
import visualization
import pyomomachsched
from shift import ShiftModel
import utility
import orders
import numpy as np
import datetime
import pandas as pd
import random
import time


# Genetic Algorithm

In [ ]:
'''order_df = orders.get_westaflex_orders()
machines = order_df["selected_machine"].astype(int).unique()
start = datetime.datetime(2022, 2, 27, 6, 0, 0)
end = datetime.datetime(2022, 3, 4, 22, 0, 0)
last_tool = "A0"
company = order_df.attrs["company_name"]
df = order_df'''

In [2]:
original_df = orders.get_westaflex_orders()
machineslist = [1531, 1532, 1533,1534, 1535, 1536, 1537,1541, 1542, 1543]
start = datetime.datetime(2022, 2, 27, 6, 0, 0)
end = datetime.datetime(2022, 3, 4, 22, 0, 0)
original_df["duration"] = (original_df["setuptime_material"] + original_df["setuptime_coil"] + original_df["duration_machine"] + original_df["duration_manual"]).dt.seconds # am Ende überprüfen ob es klappt
    #df["chosen_machine"] = [0]*len(df)
original_df["lateness"] = [0]*len(original_df)
#
original_df["calculated_setup_time"] = original_df["setup_time"] #um nachher das zu 0 werden zu lassen, welches nicht gebraucht wird
#schauen, ob ich es schaffe, den df als globale varaible zu definieren um den aufruf in average lateness zu sparen
df = original_df


In [3]:
ids=[]
for i in range (len(original_df)):
    x = len(original_df)-(i+1)
    ids.append(x)


In [ ]:
original_df.tail(n=50)

In [8]:
df = original_df.reset_index(drop=True)
df.index = ids

In [ ]:
print(ids)

In [ ]:
print(original_df)

In [ ]:
starttime = time.time()
print("Start")
makespan(ids)
endtime = time.time()
print(endtime - starttime)

In [12]:
x = earliest_deadline_first(ids)
x.tail(n=30)

,job,item,order_release,tube_type,selected_machine,machines,calculated_setup_time,tool,setuptime_material,setuptime_coil,...,calculated_start,calculated_end,planned_start,planned_end,final_start,final_end,setup_time,status,duration,lateness
160,92021730909,65PAK,2021-09-30,PAK,1531,1531,0 days 00:17:00,A0 047,0 days 00:15:00,0 days 00:02:00,...,2022-09-11 17:20:43,2022-09-11 18:00:49,NaT,NaT,NaT,NaT,0 days 00:17:00,JobStatus.UNPLANNED,2406,118 days 09:06:49
161,92021741254,75APK4200,2021-09-29,APK,1531,1531,0 days 00:17:00,A0 028,0 days 00:15:00,0 days 00:02:00,...,2022-09-18 12:55:24,2022-09-18 14:00:27,NaT,NaT,NaT,NaT,0 days 00:17:00,JobStatus.UNPLANNED,3903,128 days 04:51:27
162,92021731633,90PAPK003,2021-09-28,PAPK,1531,1531,0 days 00:17:00,A0 001,0 days 00:15:00,0 days 00:02:00,...,2022-09-25 10:00:05,2022-09-25 15:42:06,NaT,NaT,NaT,NaT,0 days 00:17:00,JobStatus.UNPLANNED,20521,135 days 06:33:06
163,92021741234,75APK1000,2021-09-27,APK,1531,1531,0 days 00:17:00,A0 028,0 days 00:15:00,0 days 00:02:00,...,2022-10-02 17:23:45,2022-10-02 18:31:08,NaT,NaT,NaT,NaT,0 days 00:17:00,JobStatus.UNPLANNED,4043,142 days 09:22:08
164,92021741249,75APK1501,2021-09-26,APK,1531,1531,0,A0 028,0 days 00:15:00,0 days 00:02:00,...,2022-10-09 21:20:10,2022-10-10 00:59:49,NaT,NaT,NaT,NaT,0 days 00:17:00,JobStatus.UNPLANNED,13179,150 days 04:10:49
165,92021730626,60APK009,2021-09-25,APK,1533,"1531,1532,1533",0,A0 035,0 days 00:15:00,0 days 00:02:00,...,2022-04-09 07:45:28,2022-04-09 12:20:40,NaT,NaT,NaT,NaT,0 days 00:17:00,JobStatus.UNPLANNED,16512,-31 days +03:26:40
166,92021640616,50AA61200,2021-09-24,AA,1535,"1531,1532,1533,1535,1536,1537",0,A0 005,0 days 00:15:00,0 days 00:02:00,...,2022-05-03 18:11:15,2022-05-03 18:59:36,NaT,NaT,NaT,NaT,0 days 00:17:00,JobStatus.UNPLANNED,2901,-3 days +22:10:36
167,92021529592,75APK001,2021-09-23,APK,1531,1531,0,A0 028,0 days 00:15:00,0 days 00:02:00,...,2022-10-17 07:28:30,2022-10-17 08:39:16,NaT,NaT,NaT,NaT,0 days 00:17:00,JobStatus.UNPLANNED,4246,159 days 23:45:16
168,92021529110,60APK009,2021-09-22,APK,1532,"1531,1532,1533",0 days 00:17:00,A0 035,0 days 00:15:00,0 days 00:02:00,...,2022-04-09 15:31:41,2022-04-10 01:34:54,NaT,NaT,NaT,NaT,0 days 00:17:00,JobStatus.UNPLANNED,36193,-31 days +16:40:54
169,92021640902,63AA0420,2021-09-21,AA,1531,1531,0 days 00:17:00,A0 038,0 days 00:15:00,0 days 00:02:00,...,2022-10-24 16:18:43,2022-10-24 17:06:05,NaT,NaT,NaT,NaT,0 days 00:17:00,JobStatus.UNPLANNED,2842,171 days 07:57:05


In [7]:
makespan(ids)

NameError: name 'makespan' is not defined

In [4]:
#resetting/initialising the dataframe using orders.py
def initializing_new_df(ids):
    #starttime = time.time()
    #print("init_new_df")
    global df
    df = original_df.reset_index(drop=True)
    df.index = ids
    df = df.sort_index(ascending=True)
    #machines = df["selected_machine"].astype(int).unique()
    
    
    #last_tool = "A0"
    #company = df.attrs["company_name"]
    

    
    return df

In [5]:
def create_helperframe():
    #starttime = time.time()
    #print("create_helperframe")
    data = {
        "machineworkload": [0] * len(machineslist),
        "jobs" : [[],[],[],[],[],[],[],[],[],[]], # manuelle Eingabe wichtig, da sonst nur Kopien von einer List erstellt werden
       }

    helperframe = pd.DataFrame(data=data,index=machineslist)
    helperframe = helperframe.reset_index(drop=True)
    helperframe.index = machineslist
    
    return helperframe

In [11]:
#weise den ersten job die erste Maschine zu die frei ist (eventuell random, oder siehe notizen)

#Problem: Füge bei dem ersten Wert das Startdatum hinzu und starte bei Wert 2, ansonsten Probleme mit calculated_start

def earliest_deadline_first(ids):
    #starttime = time.time()
    #print("Earliest_deadline")

    df = initializing_new_df(ids)
    
    
    #creating helperframe to determine the right machine which has to be used
    #
    helperframe = create_helperframe()

    #Ordering the jobs to the right machine from index 1 to end of df
    for i in range(len(df)): 
        machine = 0
        
        
        #machines zum Tuple formatieren um mimimum nehmen zu können
        included_keys = df["machines"].iloc[i].split(",")
        included_keys_list = list(map(int, included_keys ))
        included_keys_tuple = tuple(list(map(int, included_keys )))
        
        
        #machine_index
        minimum = min(helperframe.loc[included_keys_tuple,:]["machineworkload"])#berechnet die geringst Auslastung der ausgewählten Maschinen
        machines_index = helperframe.index[helperframe["machineworkload"] == minimum].tolist() # Maschine(n) die auf Basis der minimalen Auslastung gewählt wurden.List als Ausgabe
        
        
        if len(machines_index) > 1:
            machine = random.choice(machines_index)
        else:
            machine = machines_index[0]


        #Rest
        current_workload = int(helperframe.loc[machine]["machineworkload"])


        helperframe.at[machine, "machineworkload"] += df.iloc[i,:]["duration"]
        df.at[i,"selected_machine"] = machine

        if len(helperframe.loc[machine]["jobs"])>0:
            #hier müssen wir schauen welcher job zuletzt auf der maschine lief und die endzeit als startzeit wählen
            machine_jobs = helperframe.loc[machine]["jobs"] #Anzeigen der bisherigen Jobs
            last_job_id = machine_jobs[-1] # Die ID des letzten Jobs ausgeben, um den neuen an das Ende des alten anzuschließen
            df.at[i, "calculated_start"] = df.loc[last_job_id]["calculated_end"] + datetime.timedelta(seconds=current_workload)
        else:
            df.at[i, "calculated_start"] = start
        
        
        df.at[i, "calculated_end"] = df.loc[i]["calculated_start"] + datetime.timedelta(seconds=int(df.loc[i]["duration"]))
        df.at[i, "lateness"] = df.loc[i]["calculated_end"] - df.loc[i]["deadline"]

        helperframe.at[machine,"jobs"].append(i)
    
    
    for i in range(len(df)):
        
        big_helperframe = helperframe.explode("jobs")
        
        current_machine = df.at[i,"selected_machine"]
        machine_jobslist = helperframe.at[current_machine,"jobs"]
        
        current_job_on_machine_index = machine_jobslist.index(i)#
        last_tool = "start"
        if current_job_on_machine_index > 0:
            last_job_on_machine_index = machine_jobslist.index(i) - 1
            last_job = machine_jobslist[last_job_on_machine_index]
            
            last_tool = df.at[last_job, "tool"]
        else:
            
            last_tool = "no tool"
        
        
        #setting the setuptime to zero, if we do not need any setup
        if df.loc[i]["tool"] == last_tool:
            df.at[i,"calculated_setup_time"] = 0
            
        
        

    return df#hier muss helperframe für unsere heuristik stehen

In [ ]:
def clean_list(list_):
    list_ = list_.replace(', ', '","')
    list_ = list_.replace('[', '["')
    list_ = list_.replace(']', '"]')
    return list_

In [ ]:
def string_to_list(x):
    y = x.split(",")
    x = list(map(int, y))
    return x

In [ ]:
#Weise jeder Maschine die als erstes wieder frei ist den nächsten Job zu, dies ist allerdings nicht optimierbar durch einen genetischen Algorithmus, denke ich

#def machine_first(ids):
df = initializing_new_df(ids)
helperframe = create_helperframe()
helperframe = earliest_deadline_first(ids)
#print(helpef)


minimum = helperframe.machineworkload.min()


next_machine = helperframe.index[helperframe["machineworkload"] == minimum].tolist()

if len(next_machine) > 1:
    machine = random.choice(next_machine)
else:
    machine = next_machine[0]
    

helperframe.at[machine, "machineworkload"] += df.iloc[i,:]["duration"]

#df.machines = df.machines.apply(clean_list)
#df.tail(n=10)

#getting the job index
df.machines = df.machines.apply(string_to_list)
big_df = df.explode("machines")
df.tail(n=10)
print(machine)
possible_job_duration = []
for machine in machineslist:
#machine = 1532
    jobs_for_machine = big_df.job[big_df.machines == machine] # welche Jobs können auf der Maschine laufen?
    amount_of_jobs = len(jobs_for_machine)
    possible_job_duration_on_machine = jobs_for_machine.sum()
    possible_job_duration.append(possible_job_duration_on_machine)
    
    
helperframe["possible_duration"] = possible_job_duration

sorted_helperframe = helperframe.sort_values("possible_duration")
machine_order = sorted_helperframe.index

while i < len(df.jobs):
    while j < len(machineslist):
        jobs = helperframe.jobs
        jobframe = pd.DataFrame(jobs)
        #Prüfe wie viele andere Maschinen dieser Job hat
        for job in jobs:
            df.machine.iloc[job]
        
        
print(sorted_helperframe)




##die Heuristik nutzen, die ich mir auf meinen tablet aufgeschrieben habe, diese heuristik
#könnte sogar besser sein, als der genetische algorithmus

#next_machine = min(helperframe.loc[included_keys_tuple,:]["machineworkload"])
    

In [13]:
#Nur in Kombination mit makespan einsetzbar, da dort der df generiert wird, ansonsten df = earliest_deadline_first(ids) ergänzen
def average_lateness(ids):
    
    #Average lateness
    lateness_seconds = pd.to_timedelta(df["lateness"]).dt.total_seconds()
    #lösche Negative Einträge zu 0:
    lateness_seconds = lateness_seconds.apply(lambda x: x if x>=0 else 0)
    total_lateness = lateness_seconds.sum()
    average_lateness = total_lateness / len(df)
    
    return average_lateness

In [14]:
#makespan

#ich vermute es ist am sinnvollsten duration zu addieren, da vermutlich die jobs mit der längsten duration auch am wertvollsten sind
#ansonsten einfach über den helperframe die länge der joblisten nutzen
def makespan(ids):
    #starttime_deadline = time.time()
    #print("deadline")
    df = earliest_deadline_first(ids)
    #endtime_deadline = time.time()
    #global z
    #print(z)
    #z+=1
    #print(endtime_deadline - starttime_deadline)
    #print("deadline ende")
    
    
    

    zero_entries = df

    for i in range (len(df)):
        if end<zero_entries.loc[i]["calculated_end"]:

            zero_entries.at[i,"duration"] = 0

    zero_entries["duration"] - pd.to_timedelta(zero_entries["calculated_setup_time"]).dt.total_seconds()
    zero_entries["duration"].apply(lambda x: 0 if x<0 else x)
    makespan = zero_entries["duration"].sum()
    
    
    return makespan

#duration.apply(lambda x: 0 if end < df.loc[x]["calculated_end"] else x) dies könnte schneller ansatz sein, aber vielleicht nicht möglich
#die Ausgabe ist in Sekunden



In [ ]:
'''
to do:
 - die oberen Zeilen vernünftig in Funktionen packen vermutlich makespan und average lateness als 2 verschiedene
   objective functions nutzen und bei jeder neuen Generation muss die obere Formatierung benutzt werden, damit man die
   beiden functions anwenden kann
 - Einmal schauen ob man Pymoo nutzen kann und falls nicht
 - das untere Template nutzen, oder einmal bei chatgpt ein Template nutzen
 - Freitag 7,5h arbeiten, dann sind 20 Stunden voll diese Woche

'''

In [15]:
from pymoo.operators.crossover.pntx import TwoPointCrossover
from pymoo.operators.sampling.rnd import PermutationRandomSampling
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.mutation.inversion import InversionMutation
from pymoo.operators.crossover.ox import OrderCrossover
from pymoo.core.problem import ElementwiseProblem
from pymoo.operators.crossover.sbx import SBX

#z=1

id_init = list(original_df.index)
id_init_array=np.array(id_init)
starttime = time.time()
print("Start")

class MyProblem(ElementwiseProblem):

    def __init__(self):
        super().__init__(
                         n_var=190,
                         n_obj=2,
                         n_ieq_constr=0, # constrains auf zero if we dont need them 
                         )

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = -abs(makespan(x))
        f2 = average_lateness(x)


        out["F"] = [f1, f2]

'''def obj1(x):
    return -abs(makespan(x))

def obj2(x):
    
    return average_lateness(x)

problem = Problem(n_var=190, n_obj=2, objectives=[obj1, obj2])
'''

problem = MyProblem()
algorithm = NSGA2(
                    pop_size=190,
                    pop=id_init,
                    mutation=InversionMutation(),
                    #crossover=SBX(prob=0.9, eta=15),
                    crossover=OrderCrossover(),
                    sampling=PermutationRandomSampling(),
                    
                 )

from pymoo.termination import get_termination

termination = get_termination("n_gen", 1)

from pymoo.optimize import minimize

res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               display=None,
              )

X = res.X
F = res.F

endtime = time.time()
print(endtime - starttime)



Start
92.46766114234924


In [ ]:
print(res.F)

In [ ]:
print(X)

In [ ]:
#Welches X gehört zu welchem F?

## Vergleich der Algorithmen

In [ ]:
from termination import naive_termination
from pyomomachsched import opt_schedule

#df_short = original_df.drop(index=original_df.index[:180], axis=0, inplace=True)
df_short = original_df

df_short.drop(index=df.index[:80], axis=0, inplace=True)
df_short= df_short.reset_index(drop=True)

In [ ]:
print(len(df_short))

In [ ]:
display(df_short)

In [ ]:
starttime = time.time()
print("Start")
df_naive = naive_termination(df_short, datetime.datetime(2022, 2, 27, 6, 0, 0), "A0")
#df_pyomo = opt_schedule(df_short, datetime.datetime(2022, 1, 2, 0, 0, 0))
endtime = time.time()
print(endtime - starttime)


In [ ]:
df_naive.tail(n=10)

In [ ]:
def makespan_short(df):


    zero_entries = df
    
    for i in range (len(df)):
        if end<zero_entries.loc[i]["calculated_end"]:
            zero_entries.at[i,"duration"] = 0

    
    zero_entries["duration"] = zero_entries["duration"] - pd.to_timedelta(zero_entries["calculated_setup_time"]).dt.total_seconds()
    
    zero_entries["duration"].apply(lambda x: 0 if x<0 else x)
    
    makespan = zero_entries["duration"].sum()
    
    
    
    
    return makespan

In [ ]:
def avg_lateness_short(df):
    for i in range(len(df)):
        df.at[i, "lateness"] = df.loc[i]["calculated_end"] - df.loc[i]["deadline"]
    
    lateness_seconds = pd.to_timedelta(df["lateness"]).dt.total_seconds()
    #lösche Negative Einträge zu 0:
    lateness_seconds = lateness_seconds.apply(lambda x: x if x>=0 else 0)
    total_lateness = lateness_seconds.sum()
    average_lateness = total_lateness / len(df)
    return average_lateness


In [ ]:
makespan_naive = makespan_short(df_naive)
lateness_naive = avg_lateness_short(df_naive)

print("naive")
print(makespan_naive)
print(lateness_naive)



In [ ]:
makespan_pyomo = makespan_short(df_pyomo)
lateness_pyomo = avg_lateness_short(df_pyomo)

print("pyomo")
print(makespan_pyomo)
print(lateness_pyomo)

## Custom Genetic

In [ ]:
def objectiveFunctionDeadline(x):
    #Vorzeiteger Abschluss des Auftrags wird nicht positiver gewertet als genaues erreichen der Deadline
    def posfunction(x):
        if x<0:
            x=0
    return x
    amount_of_jobs = len(df.index)
    #implement a deadline missing row
    df["missed_deadline"] = (df["deadline"]-df["calculated_end"]).dt.seconds
    #if we hit a deadline the negative value should be counted as 0 for now it seems like we never miss a deadline.
    #this is suspicious and should be further investigated
    df["missed_deadline"] = df.apply(lambda row: posfunction(row['missed_deadline']), axis=1)

    return (1/amount_of_jobs)*(np.sum(df["missed_deadline"].to_numpy()))


In [ ]:
def objectiveFunctionMaschine(x):
    

In [ ]:
#objective function
def posfunction(x):
    if x<0:
        x=0
        print("yes")
    return x
amount_of_jobs = len(df.index)
#implement a deadline missing row
df["missed_deadline"] = (df["deadline"]-df["calculated_end"]).dt.seconds
#if we hit a deadline the negative value should be counted as 0 for now it seems like we never miss a deadline.
#this is suspicious and should be further investigated
df["missed_deadline"] = df.apply(lambda row: posfunction(row['missed_deadline']), axis=1)

display(df["missed_deadline"])
#display(df)
objective_function = (1/amount_of_jobs)*(np.sum(df["missed_deadline"].to_numpy()))

print(objective_function)
print((df['missed_deadline'] == 0).sum())

## Pyomo naive genetic

In [ ]:
#Genetic Algorithm Template

from random import choices, randint, randrange, random
from typing import List, Optional, Callable, Tuple

'''Genome = List[int]
Population = List[Genome]
PopulateFunc = Callable[[], Population]
FitnessFunc = Callable[[Genome], int]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunc = Callable[[Genome], Genome]
PrinterFunc = Callable[[Population, int, FitnessFunc], None]
'''


def generate_genome(length):
    return choices([0, 1], k=length)


def generate_population(size, genome_length):
    return [generate_genome(genome_length) for _ in range(size)]


def single_point_crossover(a, b):
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")

    length = len(a)
    if length < 2:
        return a, b

    p = randint(1, length - 1)
    return a[0:p] + b[p:], b[0:p] + a[p:]


def mutation(genome, num = 1, probability= 0.5):
    for _ in range(num):
        index = randrange(len(genome))
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    return genome


def population_fitness(population, fitness_func):
    return sum([fitness_func(genome) for genome in population])


def selection_pair(population, fitness_func):
    return choices(
        population=population,
        weights=[fitness_func(gene) for gene in population],
        k=2
    )


def sort_population(population, fitness_func):
    return sorted(population, key=fitness_func, reverse=True)


def genome_to_string(genome):
    return "".join(map(str, genome))


def print_stats(population, generation_id, fitness_func):
    print("GENERATION %02d" % generation_id)
    print("=============")
    print("Population: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
    print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
    sorted_population = sort_population(population, fitness_func)
    print(
        "Best: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
    print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]),
                              fitness_func(sorted_population[-1])))
    print("")

    return sorted_population[0]


def run_evolution(
        populate_func,
        fitness_func,
        fitness_limit,
        selection_func=selection_pair,
        crossover_func=single_point_crossover,
        mutation_func=mutation,
        generation_limit= 100,
        printer= None
    ):
    population = populate_func()

    for i in range(generation_limit):
        population = sorted(population, key=lambda genome: fitness_func(genome), reverse=True)

        if printer is not None:
            printer(population, i, fitness_func)

        if fitness_func(population[0]) >= fitness_limit:
            break

        next_generation = population[0:2]

        for j in range(int(len(population) / 2) - 1):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation

    return population, i

In [ ]:
print(type(df["calculated_end"]))

In [ ]:
def naive_termination(order_df, start, last_tool):
    """
    Calculates a simple termination from the given orders and returns it.

    Parameters
    ----------
    order_df: dataframe
        Orders in a dataframe containing the columns:
            machine, job, shift, order_release, duration_machine,
            setup_time
        Overwrites the values of following columns:
            calculated_start, calculated_end.
    start: datetime
        The start time of the naive termination calculation.
    last_tool: str
        The last tool name that has been used in the last job before the
        termination.

    Returns
    -------
    dataframe
        The orders with overwritten calculated_start and calculated_end.
    """
    machines = order_df["selected_machine"].astype(int).unique()
    print(machines)
    company = order_df.attrs["company_name"]
    # Für jede Maschine
    for machine in machines:
        df_machine = order_df[order_df["selected_machine"].astype(int) == machine]
        timestamp = start
        # Entsprechend der Reihenfolge timestamps berechnen
        for index, row in df_machine.iterrows():
            order_num = row["job"]
            shift = row["shift"]

            # TODO: What about already running jobs? Or jobs that are finished?
            if timestamp < row["order_release"]:
                timestamp = row["order_release"]
            # Adjust timestamp to next shift start
            shifts = ShiftModel(company, shift, timestamp)
            timestamp = shifts.get_earliest_time(timestamp)

            order_df.loc[order_df["job"] == order_num, ["calculated_start"]] = timestamp
            tool = row["tool"]
            setup_time = row["setup_time"] if tool.casefold() != last_tool.casefold() else datetime.timedelta(0)
            order_df.loc[order_df["job"] == order_num, ["calculated_setup_time"]] = setup_time
            prod_time = row["duration_machine"]
            if isinstance(prod_time, datetime.timedelta):
                prod_time = prod_time.total_seconds() / 60
            if isinstance(setup_time, datetime.timedelta):
                setup_time = setup_time.total_seconds() / 60
            runtime = prod_time - 17 + setup_time + 2
            timestamp = utility.calculate_end_time(
                start=timestamp, duration=runtime, company=company, shift=shift
            )
            order_num = row["job"]
            order_df.loc[order_df["job"] == order_num, ["calculated_end"]] = timestamp
            last_tool = tool
    return order_df


def combine_orders(order_df, start):
    """
    Combines orders with the same item properties.
    """
    # TODO: FINISH THIS FUNCTION


if __name__ == "__main__":
    # Debugging
    df = orders.get_westaflex_orders()
    df.drop(index=df.index[:180], axis=0, inplace=True)
    '''print(
        df[
            [
                "order_release",
                "machines",
                "selected_machine",
                "shift",
                "duration_machine",
                "calculated_start",
                "calculated_end",
                "setup_time",
            ]
        ]
    )'''
    df = naive_termination(df, datetime.datetime(2022, 2, 27, 6, 0, 0), "A0")
    print(
        df[
            [
                "order_release",
                "machines",
                "selected_machine",
                "shift",
                "duration_machine",
                "calculated_start",
                "calculated_end",
                "setup_time",
            ]
        ]
    )
    visualization.gantt(df)
    '''df = pyomomachsched.opt_schedule(df, datetime.datetime(2022, 1, 2, 0, 0, 0))
    with pd.option_context("display.max_rows", None, "display.max_columns", None):
        print(
            df[
                [
                    "order_release",
                    "tool",
                    "machines",
                    "selected_machine",
                    "shift",
                    "duration_machine",
                    "setup_time",
                    "calculated_start",
                    "calculated_end",
                ]
            ]
        )
    df.to_pickle("./pyomo_df.pkl")
    df.to_excel("./pyomo_df.xlsx")
    df.to_csv("./pyomo_df.csv")
    visualization.gantt(df)'''

